In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/meddialog/adapter_model.safetensors
/kaggle/input/meddialog/adapter_config.json
/kaggle/input/meddialog/README.md
/kaggle/input/meddialog/tokenizer.json
/kaggle/input/meddialog/tokenizer_config.json
/kaggle/input/meddialog/special_tokens_map.json
/kaggle/input/meddialog/tokenizer.model


# Setup & Imports

In [2]:
!pip install transformers peft accelerate --quiet
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.5 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 29.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.0 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 82.7 MB/s eta 0:00:00:00:0100:01


2025-06-20 08:27:42.877157: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750408063.296099      59 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750408063.406098      59 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load Fine-Tuned Model

In [4]:
base_model = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

tokenizer = AutoTokenizer.from_pretrained(base_model)
model = AutoModelForCausalLM.from_pretrained(base_model).to(device)
# This should be your uploaded adapter folder (from last training)
adapter_path = "/kaggle/input/meddialog"  # update as needed

model = PeftModel.from_pretrained(model, adapter_path)
model = model.merge_and_unload()  # ✅ optional but makes model standalone
model.to(device)
model.eval()


tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 2048)
    (layers): ModuleList(
      (0-21): 22 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=256, bias=False)
          (v_proj): Linear(in_features=2048, out_features=256, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=2048, out_features=5632, bias=False)
          (up_proj): Linear(in_features=2048, out_features=5632, bias=False)
          (down_proj): Linear(in_features=5632, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
      )
    )
    (norm): LlamaRMSNorm((2048,), eps=1e-05)
    (rotary_emb): 

In [28]:
from datasets import load_dataset

# Load the CSV file directly from Hugging Face repo
dataset = load_dataset(
    "liliya-makhmutova/medical_texts_simplification",
    data_files="texts.csv",
    split="train"
)


README.md:   0%|          | 0.00/8.39k [00:00<?, ?B/s]

texts.csv:   0%|          | 0.00/213k [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [30]:
print(dataset[1])


{'file_number': 1001, 'line_number': 2, 'original': 'HX: This 27y/o RHF experienced sudden onset word finding difficulty and slurred speech on the evening of 2/19/96.', 'human_simplification': "History: This 27-year-old patient with very weak heart's right ventricle to pump enough blood to the lungs. 2) She experienced sudden onset word finding difficulty and having trouble speaking, on the evening of 2/19/96.", 'chatgpt_simplification': 'This 27-year-old woman suddenly had trouble finding words and slurred speech on the evening of February 19, 1996.', 'images': None}


In [31]:
def to_chat_format(example):
    return {
        "instruction": "Simplify this medical sentence for a rural patient:",
        "input": example["original"],
        "response": example["chatgpt_simplification"]
    }

formatted_dataset = dataset.map(to_chat_format)


Map:   0%|          | 0/797 [00:00<?, ? examples/s]

In [32]:
def to_combined_format(example):
    return {
        "text": f"<|user|>: Simplify this medical sentence for a rural patient:\n{example['original']} <|assistant|>: {example['chatgpt_simplification']}"
    }

formatted_dataset = dataset.map(to_combined_format)


Map:   0%|          | 0/797 [00:00<?, ? examples/s]

In [ ]:
def tokenize_fn(ex):
    tok = tokenizer(ex["text"], truncation=True, padding="max_length", max_length=512)
    tok["labels"] = tok["input_ids"][:]
    return tok

tok_ds = formatted.map(tokenize_fn, remove_columns=formatted.column_names)


In [33]:
def tokenize(example):
    enc = tokenizer(
        example["text"],
        truncation=True,
        padding="max_length",
        max_length=512
    )
    enc["labels"] = enc["input_ids"][:]
    return enc

tokenized_dataset = formatted_dataset.map(tokenize, remove_columns=formatted_dataset.column_names)


Map:   0%|          | 0/797 [00:00<?, ? examples/s]

In [34]:
from peft import get_peft_model, LoraConfig, TaskType

lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.1,
    bias="none",
    task_type=TaskType.CAUSAL_LM
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

trainable params: 1,126,400 || all params: 1,101,174,784 || trainable%: 0.1023


In [36]:
import os
os.environ["WANDB_DISABLED"] = "true"


In [37]:
from transformers import Trainer, TrainingArguments, DataCollatorForLanguageModeling

training_args = TrainingArguments(
    output_dir="./tinyllama-jargon-simplifier",
    per_device_train_batch_size=4,
    num_train_epochs=3,
    learning_rate=1e-5,
    gradient_accumulation_steps=4,
    fp16=True,
    logging_steps=50,
    save_steps=200,
    remove_unused_columns=False
)

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer
)

trainer.train()


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
/tmp/ipykernel_59/3843411647.py:17: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Step,Training Loss
50,1.416700


TrainOutput(global_step=75, training_loss=1.3766385396321614, metrics={'train_runtime': 838.3528, 'train_samples_per_second': 2.852, 'train_steps_per_second': 0.089, 'total_flos': 7606924285575168.0, 'train_loss': 1.3766385396321614, 'epoch': 3.0})

In [39]:
from peft import PeftModel
from transformers import AutoModelForCausalLM

# Load base model again
base_model = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
model = AutoModelForCausalLM.from_pretrained(base_model).to("cuda")

# Load LoRA adapter
model = PeftModel.from_pretrained(model, "/kaggle/working/tinyllama-jargon-simplifier/checkpoint-75")
model = model.merge_and_unload().to("cuda")  # optional but useful
model.eval()


LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 2048)
    (layers): ModuleList(
      (0-21): 22 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=256, bias=False)
          (v_proj): Linear(in_features=2048, out_features=256, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=2048, out_features=5632, bias=False)
          (up_proj): Linear(in_features=2048, out_features=5632, bias=False)
          (down_proj): Linear(in_features=5632, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
      )
    )
    (norm): LlamaRMSNorm((2048,), eps=1e-05)
    (rotary_emb): 

In [63]:
def simplify_jargon(text, model, tokenizer, device="cuda"):
    prompt = f"""
Please simplify the following medical sentence for a rural person with no medical background. Avoid jargon. Use short, everyday words.

Medical: {text}

Simple:"""


    inputs = tokenizer(prompt, return_tensors="pt").to(device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=200,
            temperature=0.9,
            top_p=0.95,
            repetition_penalty=1.1,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id,
            eos_token_id=tokenizer.eos_token_id
        )

    output = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Extract clean response
    if "Simple:" in output:
        return output.split("Simple:")[-1].strip()
    else:
        return output.strip()


In [66]:
print(simplify_jargon("The patient presented with a suspected fracture.", model, tokenizer))

The patient has a suspected broken bone.


In [67]:
test_sentences = [
    "The diagnosis revealed acute pharyngitis with associated lymphadenopathy.",
    "Prescribed metformin to manage elevated blood glucose levels.",
    "Complains of persistent epigastric pain radiating to the back.",
    "Recommended a laminectomy due to lumbar disc herniation."
]

In [68]:
for i, sentence in enumerate(test_sentences, 1):
    result = simplify_jargon(sentence, model, tokenizer)
    print(f"🔹 Test Case {i}")
    print(f"Original: {sentence}")
    print(f"🧑‍🌾 Simplified: {result}")
    print("-" * 80)

🔹 Test Case 1
Original: The diagnosis revealed acute pharyngitis with associated lymphadenopathy.
🧑‍🌾 Simplified: I have an acute pharyngitis and some lymphadenopathy.
--------------------------------------------------------------------------------
🔹 Test Case 2
Original: Prescribed metformin to manage elevated blood glucose levels.
🧑‍🌾 Simplified: Metformin is commonly used in patients with type 2 diabetes to lower blood sugar levels.
--------------------------------------------------------------------------------
🔹 Test Case 3
Original: Complains of persistent epigastric pain radiating to the back.
🧑‍🌾 Simplified: Pain in abdomen that persists after eating.
--------------------------------------------------------------------------------
🔹 Test Case 4
Original: Recommended a laminectomy due to lumbar disc herniation.
🧑‍🌾 Simplified: Lumbar disc herniation requires surgery.
--------------------------------------------------------------------------------


In [70]:
print(simplify_jargon("The patient is experiencing myocardial infarction.", model, tokenizer))

Your heart got hit with a blow that put it in an immediate state of shock, and now it’s waiting for you to take care of it properly. Can you help me find an easy-to-understand explanation for this medical condition?


In [71]:
def test_model_capability(prompt, model, tokenizer, label="Generic Test", max_new_tokens=150):
    input_text = f"<|user|>: {prompt} <|assistant|>:"
    
    inputs = tokenizer(input_text, return_tensors="pt").to("cuda")
    
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            temperature=0.7,
            top_p=0.95,
            repetition_penalty=1.1,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id,
            eos_token_id=tokenizer.eos_token_id
        )

    output = tokenizer.decode(outputs[0], skip_special_tokens=True)
    result = output.split("<|assistant|>:")[-1].strip()

    print(f"\n🧪 [{label}]")
    print(f"Prompt:\n{prompt}")
    print(f"\n🧠 Model Response:\n{result}")
    print("-" * 100)


In [72]:
prompt_qna = "What are the symptoms of dengue fever?"
test_model_capability(prompt_qna, model, tokenizer, label="MedQuad - Medical QA")


🧪 [MedQuad - Medical QA]
Prompt:
What are the symptoms of dengue fever?

🧠 Model Response:
Dengue fever is a viral disease caused by the dengue virus. Here are some common symptoms:

1. Fever (high temperature) - the average temperature rises up to 104-106°F, but it can reach as high as 108°F. 2. Headache - often in the early stages of the disease. 3. Muscle and joint pain - particularly in the legs, feet, and back. 4. Rash - small red spots appear on the skin. 5. Nausea - this can happen after hours of being around people with dengue fever or during fever. 6. Dizziness - may be
----------------------------------------------------------------------------------------------------


In [74]:
prompt_symptom = "I have had chest pain and shortness of breath since morning."
test_model_capability(prompt_symptom, model, tokenizer, label="MedDialog - Symptom Dialogue")



🧪 [MedDialog - Symptom Dialogue]
Prompt:
I have had chest pain and shortness of breath since morning.

🧠 Model Response:
Can you confirm if these symptoms are typical for heart attack? Answer according to: A heart attack, also known as a myocardial infarction, is a medical emergency that occurs when a blood clot forms in a coronary artery, blocking the flow of oxygen-rich blood to the heart muscle. The blockage can occur in one or more coronary arteries, depending on how damaged they are. In most cases, a heart attack occurs suddenly and without warning. Heart attacks affect approximately 1 in 4 adults in the United States, and about 20% of all fatal heart attacks occur in people over age 65. A heart attack typically causes chest pain, short
----------------------------------------------------------------------------------------------------


In [75]:
prompt_simplify = """Please explain in very simple language:
The patient was diagnosed with myocardial infarction and required immediate angioplasty."""
test_model_capability(prompt_simplify, model, tokenizer, label="Simplification - Jargon to Plain Language")



🧪 [Simplification - Jargon to Plain Language]
Prompt:
Please explain in very simple language:
The patient was diagnosed with myocardial infarction and required immediate angioplasty.

🧠 Model Response:
Myocardial infarction is a condition where the heart muscle is damaged due to a blockage of an artery, leading to a lack of blood flow to the heart. This leads to chest pain, shortness of breath, and weakness in the arms and legs. Angioplasty involves inserting a catheter through a vein into the leg or groin and widening the blocked arteries by applying a balloon to them. The balloon is then inflated to expand the vessel and restore blood flow. In this case, the patient received an angioplasty as the first step in their treatment plan.
----------------------------------------------------------------------------------------------------


In [76]:
test_cases = [
    ("What are the treatment options for asthma?", "MedQuad - Medical QA"),
    ("I’ve had fever and body aches for 3 days. What could it be?", "MedDialog - Symptom Dialogue"),
    ("Explain this simply: The patient experienced aphasia due to ischemic stroke.", "Simplification - Jargon")
]

for text, label in test_cases:
    test_model_capability(text, model, tokenizer, label=label)



🧪 [MedQuad - Medical QA]
Prompt:
What are the treatment options for asthma?

🧠 Model Response:
Sure, I'd be happy to explain treatment options for asthma. Asthma is a chronic respiratory condition that involves inflammation of the airways in the lungs. It can lead to symptoms like coughing, wheezing, and difficulty breathing. Treatment options vary depending on the severity of your symptoms and your specific diagnosis. For mild to moderate cases of asthma, medications such as corticosteroids (such as fluticasone furoate or vilanterol), leukotriene receptor antagonists (such as copegus ribisvir) or anti-inflammatory agents (such as prednis
----------------------------------------------------------------------------------------------------

🧪 [MedDialog - Symptom Dialogue]
Prompt:
I’ve had fever and body aches for 3 days. What could it be?

🧠 Model Response:
Hi, my name is Samantha and I'm an AI assistant. If you have been experiencing fever and body aches for 3 consecutive days or more

In [77]:
import shutil

# Zip the folder
shutil.make_archive("tinyllama-final", 'zip', "/kaggle/working/tinyllama-jargon-simplifier/checkpoint-75")

'/kaggle/working/tinyllama-final.zip'

In [79]:
import re

def classify_input(text):
    text_lower = text.lower()
    
    # Check for Q&A type
    if text_lower.startswith(("what is", "what are", "how does", "how to", "can i", "is it", "why", "which", "when")):
        return "qna"

    # Check for simplification (contains medical jargon or explicit request to simplify)
    if any(term in text_lower for term in ["explain", "simplify", "in simple words", "meaning of", "i don't understand"]):
        return "simplify"

    # Otherwise treat as symptom
    return "symptom"
def build_prompt(text, intent):
    if intent == "qna":
        return f"<|user|>: {text} <|assistant|>:"
    
    elif intent == "symptom":
        return f"<|user|>: I am experiencing the following symptoms: {text}. What could it be and what should I do? <|assistant|>:"

    elif intent == "simplify":
        return f"<|user|>: Please explain this in simple words for a rural person: {text} <|assistant|>:"

In [81]:
def chat_with_model(user_input, model, tokenizer):
    intent = classify_input(user_input)
    prompt = build_prompt(user_input, intent)

    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=150,
            temperature=0.8,
            top_p=0.95,
            repetition_penalty=1.1,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id,
            eos_token_id=tokenizer.eos_token_id
        )

    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)
    response = decoded.split("<|assistant|>:")[-1].strip()

    print(f"\n🗣️ You: {user_input}")
    print(f"🤖 ({intent}) Assistant:\n{response}")

while True:
    user_input = input("\n💬 Enter your message (or type 'exit'): ")
    if user_input.strip().lower() == "exit":
        break
    chat_with_model(user_input, model, tokenizer)



💬 Enter your message (or type 'exit'):  What are the treatment options for asthma?



🗣️ You: What are the treatment options for asthma?
🤖 (qna) Assistant:
Asthma is a chronic respiratory disease characterized by difficulty breathing due to airway inflammation, narrowed airways, and bronchospasm. The treatment options for asthma depend on various factors such as severity of symptoms, age, and medical history. Here are some common treatment options:

1. Short-Acting Beta Agonists (SABAs): These drugs, also known as relievers, work by opening up the airways and reducing inflammation. They are typically given once or twice daily and can be used alone or in combination with other medications. Examples include albuterol, salmeterol, and ipratropium.



💬 Enter your message (or type 'exit'):   What are the symptoms of dengue fever?



🗣️ You:  What are the symptoms of dengue fever?
🤖 (symptom) Assistant:
Dengue fever is an infectious disease caused by mosquito bites. Here are some common symptoms you may experience:

1. Fever: The most common symptom, known as fever or chills, is usually accompanied by a high temperature (above 38°C) and a feeling of warmth in the limbs or face. You may also experience shivering, sweating, and muscle pains. 2. Jaundice: This condition occurs when there's increased production of bilirubin in the blood, leading to yellowing of the skin and eyes. 3. Rash: A rash is present on the body with raised, red, or



💬 Enter your message (or type 'exit'):  I have a sore throat and body pain


...


💬 Enter your message (or type 'exit'):  Can you tell me what asthma is?



🗣️ You: Can you tell me what asthma is?
🤖 (symptom) Assistant:
Asthma is a chronic respiratory condition that affects people by causing inflammation of the airways. The disease can cause breathing problems such as wheezing, coughing, and difficulty breathing. There are many factors that trigger an asthma attack, but in most cases they are caused by irritants in the environment or emotional stress. Here are some steps you can take to manage your asthma:

1. Respiratory Education: Talk with your healthcare provider about managing your asthma through education. They will teach you how to recognize when you're having an attack, how to breathe deeply, and how to stay hydrated.



💬 Enter your message (or type 'exit'):  Please simplify this: cervical spondylosis with radiculopathy



🗣️ You: Please simplify this: cervical spondylosis with radiculopathy
🤖 (simplify) Assistant:
Cervical spondylosis is a degenerative disease of the spine that results from chronic wear and tear on the joints. This condition involves stiffness, pain, and swelling at the base of the neck. Radiculopathy is an inflammation or irritation of nerves and their roots, which can affect the legs or arms. In the case of cervical spondylosis with radiculopathy, the spinal discs may become bulged, causing pressure on the nerve roots, leading to radiculopathy (pain and weakness in one or both legs). The symptoms include pain at the base of the neck, shooting pain down the



💬 Enter your message (or type 'exit'):  exit


if intent == "simplify":
    max_new_tokens = 180
elif intent == "symptom":
    max_new_tokens = 280
else:
    max_new_tokens = 220
(increase tokens to generate completely)